In [4]:
import math
import numpy as np
import Robothym 
import matplotlib
import os
import sys
import math
from statistics import mean
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [49]:
import os
import sys
import math
from statistics import NormalDist, mean
import pandas as pd
import numpy as np
#import tdmclient.notebook

kr = 1
ka = 15
kb = -1.5
        #thymio default parameters and thresholds
wheel_radius = 0.02
wheel_length = 0.13 #distance of wheels from each other [cm] 
max_speed = 500   #maximum thymio wheel speed
pos_th = 0.001    #
class Robothym:

    def _init_(self, lspeed=0, rspeed=0, curr_pos=(0,0,0)):
        '''
        Constructer of the Thymio Robot which contains following parameters
        lspeed : left wheel motor speed, as default set to 100
        rspeed : right wheel motor speed, as default set to 100
        curr_pos : current position, as initial it taken as (0,0,0) indicating (x,y,q)
        '''
        self.set_params()
        #setting wheel speeds
        self.phiL = lspeed
        self.phiR = rspeed
        #position and trajectory parameters
        self.curr = curr_pos
        self.node_index = 0
        self.next = None
        #Astolfi parameters
        self.rho   = None
        self.alpha = None
        self.beta  = None

        #obstacle avoidance threshold
        self.obs_th = 500 
        #astolfi controller parameters


    def set_params(self):
        self.kr = kr
        self.ka = ka
        self.kb = kb
        #thymio default parameters and thresholds
        self.wheel_radius = wheel_radius
        self.wheel_length = wheel_length #distance of wheels from each other [cm] 
        self.max_speed = max_speed   #maximum thymio wheel speed
        self.pos_th = pos_th    #

    def set_astolgain(self,astolfi_param):
        self.kr = astolfi_param[0]
        self.ka = astolfi_param[1]
        self.kb = astolfi_param[2]
        
    def get_astolfi(self):
        return self.kr, self.ka, self.kb
        
    def set_speed(self, lspeed, rspeed):
        #setting motor speeds to given parameters
        self.phiL = lspeed
        self.phiR = rspeed
    
    #setting the sampling time
    def set_steptime(self, Ts):
        self.Ts = Ts

    #givin global path, the trajectory to thymio
    def set_globalpath(self, globpath):
        self.traj = globpath
        self.next = globpath[0]

    def setCurrPos(self, curr):
        #setting current position to given value
        self.curr=curr

    def getCurrPos(self):
        #returning current position
        return self.curr

    def turn(self, angle, coords):
            R = np.array(((np.cos(angle), -np.sin(angle)),
                  (np.sin(angle),  np.cos(angle))))

    def setAstolfiA(self):
        self.A_ast = np.array((-self.kr,    0,          0),
                                (0, -self.ka+self.kr, -self.kb),
                                 0, -self.kr,            0)

    #finding speeds in polar coordinates
    def get_rho_alpha_beta_dot(self):
        return np.array(-self.kr*self.rho*math.cos(self.alpha),                             #rho_dot
                        -self.kr*math.sin(self.alpha)-self.ka*self.alpha-self.kb*self.beta, #alpha_dot
                        -self.kr*math.sin(self.alpha) )                                     #beta_dot

    #saturating inputs incase wheel speeds exceeds maximum speed
    #checking the limit of inputs before giving to Thymio
    def check_limits(self):
        #left wheel
        self.phiL =  self.max_speed if self.phiL > self.max_speed else self.phiL #larger then 500
        self.phiL = -self.max_speed if self.phiL <-self.max_speed else self.phiL #smaller than -500
        #right wheel  
        self.phiR =  self.max_speed if self.phiR > self.max_speed else self.phiR #larger than -500
        self.phiR = -self.max_speed if self.phiR <-self.max_speed else self.phiR #smaller than -500 


    def ikine(self):
        #finding polar coordinates for Astolfi controller
        #dx, dy, theta
        delta=self.compute_dist(self.curr,self.next)
        theta=self.curr[2]
        print(delta)
        #polar coordinates alpha, beta, rho
        self.rho = math.sqrt(delta[0]**2 + delta[1]**2)
        self.alpha = -theta + math.atan2(delta[1],delta[0])
        self.beta = -theta - self.alpha
        #velocities v and w
        vel = self.kr*self.rho
        omega = self.ka*self.alpha + self.kb*self.beta
        #conversion to cartesian speeds
        x_dot = vel*math.cos(theta)
        y_dot = vel*math.sin(theta)
        theta_dot = omega
        
        #finding corresponding wheel speeds
        phi_L = (x_dot*math.cos(theta) + y_dot*math.sin(theta) + self.wheel_length*omega)/self.wheel_radius
        phi_R = (x_dot*math.cos(theta) + y_dot*math.sin(theta) - self.wheel_length*omega)/self.wheel_radius
        
        self.setCurrPos(self.curr+Ts*(x_dot,y_dot,theta_dot))

    #checking if we reached the next goal
    def check_pos(self):
        delta = self.compute_dist(self.curr, self.next)
        if  delta < self.pose_th:
            if self.compute_dist(self.next,self.set_globalpath[-1]) < 0.001:
                self.node_index = -1
                self.next = self.set_globalpath[-1]
                return 0
            else: 
                self.node_index=+1  
                self.next = self.set_globalpath[self.node_index]
                delta = self.compute_dist(self.curr, self.next) 
        return delta 
            
    #computing delta
    def compute_dist(self, curr, next):
        return (next[0]-self.curr[0], next[1]-self.curr[1])
        

In [50]:
thym=Robothym()
thym.setCurrPos((0,0,0))
glob=[(1,2),(3,4),(7,8)]
astolfi_param=[1,15,-1.5]
thym.set_globalpath(glob)
x=thym.getCurrPos()
thym.set_params()
thym.set_astolgain(astolfi_param)
ka,kr,kb = thym.get_astolfi()
print(x)
print(ka,kr,kb)

(0, 0, 0)
1 15 -1.5


In [51]:
flag=True
x=[]
x.append(thym.getCurrPos())
i=0
Ts=0.001
while flag & i<100:

    thym.ikine()
    if thym.check_pos()==0: flag=False
    x.append(thym.getCurrPos())
    i+=1

(1, 2)


TypeError: can't multiply sequence by non-int of type 'float'